<a href="https://colab.research.google.com/github/ansh-tandon/Fashion-MNIST/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FASHION MNIST DATASET 
## AUTHOR: Ansh Tandon

In [1]:
!git clone https://github.com/ansh-tandon/Fashion-MNIST.git

Cloning into 'Fashion-MNIST'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 732 bytes | 732.00 KiB/s, done.


In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.11.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
#Convert the images to 0 and 1
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
#usually the images are in 3D format dimensions(height,width,channels) but see
train_images[0].shape
#so we need to reshape the Fashion-MNIST dataset to have an additional dimension for the channels, since CNNs expect image data to be in the form of (height, width, channels). In the case of the Fashion-MNIST dataset, since the images are grayscale, we can represent them with a single channel.

(28, 28)

In [ ]:
#reshape the model
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
#In order to create a CNN model make a Sequential Layer and add Conv2d LAYER and other hidden layers
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #relu returns the input if positive(leaves positive values unchanged) and 0 if negative and is effective for image classification tasks
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
     #flattening a layer is imp to get the output in 1D array as we want to pass the output to the FUully connected layer
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
   #SOFTMAX return the graph of probability distribution of taken input values 
    keras.layers.Dense(10, activation='softmax')
  ])
  #AdAM OPTIMIZER used for minimizing the loss function
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
         #Sparse entropy for generally image classification problems      
              metrics=['accuracy'])
  
  return model

In [ ]:
# Using RandomSearch to find out which Hyperparameter is best for MNIST dataset
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-61ed745f13fd>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8798333406448364

Best val_accuracy So Far: 0.9071666598320007
Total elapsed time: 00h 03m 14s


In [ ]:
#In order to get the top most model performing
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 48)        1248      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        20784     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 112)               2602096   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2,625,258
Trainable params: 2,625,258
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Retrain the model using the best model
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=8)

Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1086 - accuracy: 0.9592 - val_loss: 0.2655 - val_accuracy: 0.9165
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0756 - accuracy: 0.9713 - val_loss: 0.3587 - val_accuracy: 0.9080


In [ ]:
# saving thee file in h5 model